# MCP Agent

This cookbook implements a **MCP Agent** with **medium reasoning effort** using the **Seed1.8 model**. It integrates **external tools** (e.g., GitHub API) to enhance retrieval and summarization, enabling complex tasks via **tool chaining** and **closed-loop** inference.

### Core Features

1. Single-Turn Inference
- Processes a single user query and provides a response.
- Supports tool integration for online search and content summarization.

2. Multi-Turn Inference
- Handles multi-step reasoning to accomplish complex tasks.
- Manages conversation history using a memory condenser to optimize performance.

3. Memory Management
- Implements a FIFO-based memory condenser to retain the most appropriate conversation history during multi-turn inference.

4. Force Answer
- When max steps is reached, the agent will be forced to provide an answer.


### 0. Prerequisites
- API Key: [Obtain an API key](https://console.volcengine.com/ark/region:ark+cn-beijing/apiKey) for the Seed1.8 model.
- Model Endpoint: Ensure the base URL of the Ark API.
- Activate the Model: Activate the Seed1.8 model in [the Ark Console](https://console.volcengine.com/ark/region:ark+cn-beijing/openManagement).
- Install Dependencies: Install required Python packages.


In [ ]:
pip install -r ../requirements.txt

In [ ]:
# Copyright 2025 Bytedance Ltd. and/or its affiliates.
# SPDX-License-Identifier: Apache-2.0

# Please set the API key here
import os

os.environ['ARK_API_KEY']  = 'your_ark_api_key'
os.environ['ARK_MODEL_ENDPOINT'] = "doubao-seed-1-8-251215"
base_url = "https://ark.cn-beijing.volces.com/api/v3"

### 1. Tool Schemas

 Register external tools (e.g., search tool).




In [3]:
# tool schemas
tool_schemas = [
    {
      "type": "function",
      "function": {
            "name": "Search",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Search query"
                    }
                },
                "required": [
                    "query"
                ]
            },
            "description": "This is an online search tool. Input a search query, and it returns a list of web pages with corresponding summary information."
      }
  },
]

### 2. Code Setup
- **Prepare Inputs**: System prompts + user instructions in Ark Chat format.
- **Model API Configuration**: Set `api_key`, `base_url`, and `model` endpoint.

In [9]:
import json
import os
import requests

def inference(messages, tool_schemas=[]):

    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }

    data = {
        "model": model,
        "messages": messages,
        "max_tokens": 4096,
        "top_p": 0.9,
        "tools": tool_schemas,
        "temperature": 0.0,
        "reasoning_effort": "medium"
    }
    
    response = requests.post(base_url, headers=headers, json=data)
    try:
        print(f"Response: {response.json()}")
    except:
        print("Error fetching response.json:")
        print(response)
        
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status code {response.status_code}")
        print(response.json())
        return {
            "error": f"Request failed with status code {response.status_code}",
            "details": response.text
        }
        
base_url = "https://ark.cn-beijing.volces.com/api/v3/chat/completions"
api_key = os.getenv('ARK_API_KEY')
model = os.getenv('ARK_MODEL_ENDPOINT')

messages = [
    {
        "role": "system",
        "content": "You are a helpful agent that uses tools iteratively to complete the user's task, \nand when finished, provides the final answer or simply states \"Task completed\" without further tool calls."
    },
    {
        "role": "user",
        "content": "Search weather in Beijing"
    }
]

### 3. Post-Processing
After model inference, the response is printed as JSON, and each returned tool_call is parsed—extracting the function name and JSON‑deserialized arguments—and consolidated into an actions list for downstream tool execution.

In [10]:
# Model inference
response = inference(messages, tool_schemas)

# Output model prediction results
print("\n" + "=" * 40)
print("🌟 🌟 🌟  **Model Prediction**  🌟 🌟 🌟")
print("=" * 40)
print(json.dumps(response, indent=4))  # Format JSON output

# Parse tool calls
actions = []
print("\n" + "=" * 40)
print("🔧 **Tool Calls** 🔧")
print("=" * 40)
for tool_call in response["choices"][0]["message"]["tool_calls"]:
    print("\n📌 Tool Call:")
    print(json.dumps(tool_call, indent=4, ensure_ascii=False))  # Format JSON output
    function_name = tool_call["function"]["name"]
    function_arguments = tool_call["function"]["arguments"]
    function_arguments = json.loads(function_arguments)
    actions.append(
        {
            "action_type": function_name,
            "action_inputs": function_arguments
        }
    )

Response: {'choices': [{'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None, 'message': {'content': '', 'reasoning_content': 'Got it, let\'s see. The user wants to search the weather in Beijing. I need to use the Search function here. The query should be "weather in Beijing". Let me format that correctly with the function call tags.', 'role': 'assistant', 'tool_calls': [{'function': {'arguments': '{"query":"weather in Beijing"}', 'name': 'Search'}, 'id': 'call_yieh27idnkglk806mabrl3j6', 'type': 'function'}]}}], 'created': 1765458535, 'id': '021765458533952670487d793dd7e908e44567c9af90743d4adaf', 'model': 'doubao-seed-1-8-preview-251115', 'service_tier': 'default', 'object': 'chat.completion', 'usage': {'completion_tokens': 74, 'prompt_tokens': 402, 'total_tokens': 476, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 47}}}

🌟 🌟 🌟  **Model Prediction**  🌟 🌟 🌟
{
    "choices": [
        {
            "finish_reason": "tool_calls",
 